# Binh and Korn function

---
Description:

- Optimization (min)
- Multi-objective (2)
- Constraints (2)
---


The goal is to **minimize** the following equations:

\begin{cases}
      f_{1}\left(x,y\right) = 4x^{2} + 4y^{2}, \\
      f_{2}\left(x,y\right) = \left(x - 5\right)^{2} + \left(y - 5\right)^{2} \\
\end{cases}

subject to constraints:

\begin{cases}
      C_{1}\left(x,y\right) = \left(x - 5\right)^{2} + y^{2} \leq 25, \\
      C_{2}\left(x,y\right) = \left(x - 8\right)^{2} + \left(y + 3\right)^{2} \geq 7.7 \\
\end{cases}

where:

\begin{cases}
      0\le x \le 5, \\
      0\le y \le 3. \\
\end{cases}

The Pareto-optimal solutions are constituted by solutions:
    $ x=y ∈ [0.0, 3.0] \text{  and  } x ∈ [3.0, 5.0], y=3.0.$

### First we import python libraries and set up the directory of our code.

In [3]:
import os, sys
import numpy as np
from math import fsum

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [5]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.random_mutator import RandomMutator

### Define the multi-objective function, which plays also the role of the 'fitness' function.

In addition, we define the 'rand_fx', 'rand_fy' which take the role of the 'random()' method of the Genes.
Every time we want to 'mutate' a gene this function will be called and assign a 'valid', but random value
for the gene.

In [7]:
# Multi-objective function.
def fun_Binh_Korn(individual: Chromosome, f_min: bool = True):
    
    # Set the penalty coefficient.
    p_c = 0.5
    
    # Extract the data values as 'x' and 'y', for parsimony.
    x, y = [gene.value for gene in individual.genome]
    
    # Compute each objective function.
    f1 = 4.0 * (x**2 + y**2)
    f2 = (x - 5.0)**2 + (y - 5.0)**2
    
    # Compute the constraints.
    C1 = max(0.0, (x - 5.0)**2 + y**2 - 25.0)**2
    C2 = min(0.0, (x - 8.0)**2 + (y + 2.0)**2 - 7.7)**2
    
    # Compute the final value.
    f_val = fsum([0.5*f1, 0.5*f2, p_c*C1, p_c*C2])
    
    # Return the negative (to account for minimization).
    return -f_val if f_min else f_val
# _end_def_

# Random functions:
rand_fx = lambda: np.random.uniform(0.0, 5.0)
rand_fy = lambda: np.random.uniform(0.0, 3.0)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case each
gene has different random() function (set by 'rand_fx' and 'rand_fy'), because they are valid in different ranges
according to the problem definition.

In [9]:
# Define the number of chromosomes.
N = 100

# Initial population.
population = [Chromosome([Gene(np.random.uniform(0.0, 5.0), rand_fx),
                          Gene(np.random.uniform(0.0, 3.0), rand_fy)], np.nan, True)
              for _ in range(N)]

# Create the StandardGA object that will carry on the optimization.
test_GA = StandardGA(initial_pop=population, fit_func=fun_Binh_Korn,
                     select_op=LinearRankSelector(),
                     mutate_op=RandomMutator(),
                     crossx_op=UniformCrossover())

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [11]:
test_GA(epochs=500, elitism=True, f_tol=1.0e-6, verbose=False)

Initial Avg. Fitness = -33.2080.
Final   Avg. Fitness = -29.8295.
Elapsed time: 0.752 seconds.


In [12]:
# Extract the data values as 'x' and 'y', for parsimony.
x, y = [gene.value for gene in test_GA.best_chromosome().genome]

# Compute the final objective functions.
f1 = 4.0 * (x**2 + y**2)
f2 = (x - 5.0)**2 + (y - 5.0)**2

# Print the resutls.
print(f"x={x:.5f}, y={y:.5f}", end='\n\n')
print(f"f1(x, y) = {f1:.5f}")
print(f"f2(x, y) = {f2:.5f}")

x=1.02992, y=0.99747

f1(x, y) = 8.22275
f2(x, y) = 31.78176


**Note that the above solution is indeed in the Pareto-optimal solutions!**

### End of file